In [8]:
import pandas as pd
import numpy as np

file_path = r"../data/at-dataset/Scats_Data.csv"

df = pd.read_csv(file_path, sep="\t")

df[["Detector_ID", "Lane"]] = df["Detector"].str.split("-", expand=True)
df = df.drop(columns=["Detector"])
df["Detector_ID"] = pd.to_numeric(df["Detector_ID"], errors="coerce")
df["Lane"] = pd.to_numeric(df["Lane"], errors="coerce")
df["DateTime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format="%Y-%m-%d %H:%M")

df = df.drop_duplicates(subset=["Detector_ID", "Lane", "DateTime"])
# df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
# df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time

In [9]:
df.describe()

,Invalid,Volume,Detector_ID,Lane,DateTime
count,1.268714e+06,1.268714e+06,1.268714e+06,1.268714e+06,1268714
mean,5.868234e-01,1.241695e+02,2.184701e+03,7.205200e+00,2024-04-14 22:37:38.667910400
min,0.000000e+00,0.000000e+00,2.013000e+03,1.000000e+00,2023-01-01 00:00:00
25%,0.000000e+00,1.600000e+01,2.061000e+03,3.000000e+00,2023-08-23 20:00:00
50%,0.000000e+00,7.200000e+01,2.257000e+03,6.000000e+00,2024-04-14 06:00:00
75%,0.000000e+00,1.920000e+02,2.257000e+03,1.000000e+01,2024-12-03 03:00:00
max,2.400000e+01,1.498000e+03,2.402000e+03,1.900000e+01,2025-07-31 23:00:00
std,2.590651e+00,1.389751e+02,1.396438e+02,4.670805e+00,NaN


In [ ]:
# ## check missing entry
# full_time_index = pd.date_range(
#     start=df["DateTime"].min(),
#     end=df["DateTime"].max(),
#     freq="h"
# )

# missing_summary = {}

# for (det, lane), group in df.groupby(["Detector_ID", "Lane"]):
#     group = group.set_index("DateTime").sort_index()
#     # Drop duplicate DateTime entries to avoid reindex error
#     group = group[~group.index.duplicated(keep='first')]
#     reindexed = group.reindex(full_time_index)
    
#     n_missing = reindexed["Volume"].isna().sum()
#     if n_missing > 0:
#         missing_summary[(det, lane)] = n_missing


# missing_df = pd.DataFrame.from_dict(
#     missing_summary, orient="index", columns=["MissingEntries"]
# ).reset_index()
# missing_df.rename(columns={"level_0": "DetectorID", "level_1": "Lane"}, inplace=True)

# print(missing_df.sort_values("MissingEntries", ascending=False))

         index  MissingEntries
49   (2322, 6)             459
48   (2322, 5)             459
47   (2322, 4)             459
46   (2322, 3)             459
45   (2322, 2)             459
44   (2322, 1)             459
43  (2257, 19)             459
42  (2257, 18)             459
41  (2257, 17)             459
40  (2257, 16)             459
39  (2257, 15)             459
38  (2257, 14)             459
37  (2257, 13)             459
36  (2257, 12)             459
35  (2257, 11)             459
34  (2257, 10)             459
33   (2257, 9)             459
32   (2257, 8)             459
31   (2257, 7)             459
30   (2257, 6)             459
29   (2257, 5)             459
28   (2257, 4)             459
27   (2257, 3)             459
26   (2257, 2)             459
25   (2257, 1)             459
8    (2013, 9)             435
7    (2013, 8)             435
6    (2013, 7)             435
5    (2013, 6)             435
4    (2013, 5)             435
3    (2013, 4)             435
2    (20

In [11]:
df.groupby(["Detector_ID", "Lane"]).size()

Detector_ID  Lane
2013         1       22197
             2       22197
             3       22197
             4       22197
             5       22197
             6       22197
             7       22197
             8       22197
             9       22197
             10      22197
             11      22197
             12      22197
             13      22197
2061         1       22412
             2       22412
             3       22412
             4       22412
             5       22412
             6       22412
             7       22412
             8       22412
             9       22412
             10      22412
             11      22412
             12      22412
2257         1       22173
             2       22173
             3       22173
             4       22173
             5       22173
             6       22173
             7       22173
             8       22173
             9       22173
             10      22173
             11      22173
          

In [ ]:
site_list = df['Detector_ID'].unique()
main_df = pd.DataFrame()
for site in site_list:
    lane_list = df[df['Detector_ID'] == site]['Lane'].unique()
    df_lane_list = pd.DataFrame()
    for lane in lane_list:
        full_time_index = pd.date_range(
            start=df["DateTime"].min(),
            end=df["DateTime"].max(),
            freq="h"
        )

        interpolate_df = pd.DataFrame(index=full_time_index)
        interpolate_df = interpolate_df.reset_index().rename(columns={'index': 'DateTime'})
        interpolate_df['Detector_ID'] = site
        interpolate_df['Lane'] = lane
        interpolate_df['Traffic_Volume'] = df[(df['Detector_ID'] == site) & (df['Lane'] == lane)]['Traffic_Volume'].values
        interpolate_df.sort_values('DateTime')
        interpolate_df = interpolate_df.interpolate(method='time')
        df_lane_list = pd.concat([df_lane_list, interpolate_df])
    main_df = pd.concat([main_df, df_lane_list])



    # site_data = df[df['Detector_ID'] == site]
    # site_data.sort_values('DateTime')
    # site_lane1 = site_data[site_data['Lane'] == 1]
    # site_lane1 = site_lane1.set_index('DateTime').sort_index()



    # missing_df = pd.DataFrame(full_time_index, columns=['DateTime'])
    # missing_df['Missing'] = missing_df['DateTime'].apply(lambda x: x not in site_lane1.index)
    # missing_date = missing_df[missing_df['Missing'] == True]['DateTime']
    # missing_date = pd.DataFrame(missing_date)

    # missing_date['Date'] = missing_date['DateTime'].dt.date
    # missing_date.groupby('Date').size()
    # # store missing date into csv
    # missing_date.to_csv(f'../data/at-dataset/missing_dates_{site}.csv', index=False)



Site: 2061, Lane Number: [ 7  4 12  2 11  3  6  1  8 10  9  5]
Site: 2402, Lane Number: [6 3 8 7 4 1 2]
Site: 2257, Lane Number: [ 6  4  3  8  9 18 17  5 13 16 10  2 15  7 12 14  1 19 11]
Site: 2013, Lane Number: [ 8 13  9  4  2  3  7 12  5 10  1 11  6]
Site: 2322, Lane Number: [5 3 4 2 6 1]


In [43]:
site_list = df['Detector_ID'].unique()
full_time_index = pd.date_range(df["DateTime"].min(), df["DateTime"].max(), freq="h")

main_df = pd.DataFrame()

for site in site_list:
    lane_list = df.loc[df['Detector_ID'] == site, 'Lane'].unique()
    df_lane_list = pd.DataFrame()
    for lane in lane_list:
        # base hourly frame for this site/lane
        interpolate_df = (
            pd.DataFrame(index=full_time_index)
            .rename_axis('DateTime')
            .reset_index()
        )
        interpolate_df['Detector_ID'] = site
        interpolate_df['Lane'] = lane

        # take the actual observations for this site/lane
        sub = df[(df['Detector_ID'] == site) & (df['Lane'] == lane)][
            ['DateTime', 'Volume']
        ].copy()

        # merge to align values to the hourly grid
        interpolate_df = interpolate_df.merge(
            sub, on='DateTime', how='left'
        )

        # set DateTime index for time-aware interpolation
        interpolate_df = interpolate_df.sort_values('DateTime').set_index('DateTime')

        # time-aware interpolation (add a limit if you want only short gaps filled)
        interpolate_df['Volume'] = interpolate_df['Volume'].interpolate(
            method='time'  #, limit=6
        )

        # back to rows
        interpolate_df = interpolate_df.reset_index()

        df_lane_list = pd.concat([df_lane_list, interpolate_df], ignore_index=True)

    main_df = pd.concat([main_df, df_lane_list], ignore_index=True)

